<a href="https://colab.research.google.com/github/chanmol1999/LamenessDetection/blob/main/dryad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install skrebate
import numpy as np
from skrebate import ReliefF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, make_scorer, f1_score
from sklearn.model_selection import cross_val_predict

  Created wheel for skrebate: filename=skrebate-0.62-cp37-none-any.whl size=29256 sha256=1d9c11d441a2dd661346b8b7c5b785f914f4f8828fc279e693b66f4bb1a34ee5
  Stored in directory: /root/.cache/pip/wheels/6b/0e/2f/540392932d3a01060f04c242d3b9bee80dfd87f63d80b3a9a0
Successfully built skrebate


In [ ]:
#%% scoring functions
def tn(y_true, y_pred):    
  return confusion_matrix(y_true, y_pred)[0, 0]

In [ ]:
def fp(y_true, y_pred):
  return confusion_matrix(y_true, y_pred)[0, 1]

In [ ]:
def fn(y_true, y_pred):
  return confusion_matrix(y_true, y_pred)[1, 0]

In [ ]:
def tp(y_true, y_pred):
  return confusion_matrix(y_true, y_pred)[1, 1]

In [ ]:
#%% read in & pre-process featureset
# specify file path & read file
featureset2018_p = 'dataset.csv'
fs2018 = pd.read_csv(featureset2018_p)

# generate binary Lameness column
fs2018.loc[fs2018.LMS < 2, 'Lameness'] = 0
fs2018.loc[fs2018.LMS > 1, 'Lameness'] = 1
print(fs2018)

       Unnamed: 0  LMS  ...  SpectralArea_GyrMag_diff  Lameness
0               1    3  ...               2921935.083       1.0
1               2    3  ...               4154000.541       1.0
2               3    3  ...               5093477.749       1.0
3               4    3  ...               2192810.351       1.0
4               5    3  ...               2290385.773       1.0
...           ...  ...  ...                       ...       ...
20099       20100    0  ...               5052865.902       0.0
20100       20101    0  ...               4262954.212       0.0
20101       20102    0  ...               3226348.909       0.0
20102       20103    0  ...               3107034.891       0.0
20103       20104    0  ...               4635145.117       0.0

[20104 rows x 393 columns]


In [ ]:
#%% Define Label & Select relevant columns: AccMag_g_dff & GyrMag_diff
# initialise column list
diff_feature_columns = ['SheepID', 'RecordID', 'Day', 'WalkingStandingLying', 'Lameness']
# only use columns that are based on AccMag_G and GyrMag Diff columns
for col in fs2018.columns:    
  if (('AccMag_g' in col) or ('GyrMag' in col)) & ('diff' in col):        
    diff_feature_columns.append(col)

In [ ]:
# filter featureset
RelevantSubset = fs2018[diff_feature_columns]
#%% variables for classificaiton
# diverse range of models
models = [RandomForestClassifier(n_estimators=250, class_weight="balanced"),
          SVC(), MLPClassifier(), KNeighborsClassifier(5),
          AdaBoostClassifier(n_estimators=100)]
names = ['RF', 'SVM', 'MLP', 'KNC', 'AdaBoost']
# measures to extract
measures = ['accuracy', 'precision', 'recall', 'f1', 'specificity']
# stratified 10-fold cross-validation
skf = StratifiedKFold(10, shuffle=True)
# specify scoring functions to use
scoring = {'accuracy': make_scorer(accuracy_score),'precision': make_scorer(precision_score),'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score),'tp' : make_scorer(tp),'tn' : make_scorer(tn),'fp' : make_scorer(fp),'fn' : make_scorer(fn)}


In [ ]:
# %% initialise result DF
# result df column names
colnames = ['WSL', 'Features']
for col in [f'{i}_{measure}' for i in names for measure in measures]:    
  colnames.append(col)
#%% Classifier Pipeline -  ReleiefF
# initialise result df
Train = pd.DataFrame(columns=colnames)
Test = pd.DataFrame(columns=colnames)

In [ ]:
# iterate over activities
for wsl in ['Walking','Standing','Lying']:
  print(wsl)     
# filter featureset for relevant behaviour   
# print(wsl) 
  RSwsl = RelevantSubset[RelevantSubset.WalkingStandingLying == wsl].copy()    
  Xwsl = RSwsl.drop(['SheepID', 'RecordID', 'Day', 'WalkingStandingLying', 'Lameness'],axis=1).copy()
  y = RelevantSubset[RelevantSubset.WalkingStandingLying == wsl].Lameness.copy()
#  Feature Ranking   
  print(RSwsl) 
  print('apply ReliefF') 
  print(y)   
  RelF = ReliefF()
# labels = np.unique(y); 
# print(labels)    
  RelF.fit(Xwsl.values, y.values)    
  Feature_Ranking = Xwsl.columns[RelF.top_features_]
  print(Feature_Ranking)
  for nf in range(1,33):
    print(f'{nf} feature(s)')
    print(Feature_Ranking[:nf])        
    X = Xwsl[Feature_Ranking[:nf]]        
    # model building        
    training = [wsl, nf]        
    testing = [wsl, nf]        
    for est, name in zip(models, names):            
      print(name)           
      cv_res = cross_validate(est, X, y, cv=skf,                                    
                              scoring=scoring, return_train_score=True)            
      RSwsl[f'{name}_predictions_{nf}'] = cross_val_predict(est, X, y, cv=skf)            
      # store cv results in result vector            
      print('append results')            
      for measure in measures:        
        if measure == 'specificity':                    
          continue                
        training.append(cv_res[f'train_{measure}'].mean())                
        testing.append(cv_res[f'test_{measure}'].mean())            
      training.append((cv_res['train_tn'] / (cv_res['train_tn'] + cv_res['train_fp'])).mean())            
      testing.append((cv_res['test_tn'] / (cv_res['test_tn'] + cv_res['test_fp'])).mean())        
    Train.loc[len(Train)] = training
    Test.loc[len(Test)] = testing        
    print()   
  RSwsl.to_csv(wsl+'rswl.csv', index=False)    
  print()
Train.to_csv('train.csv', index=False)
Test.to_csv('test.csv', index=False)
print(cv_res)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


append results
KNC
append results
AdaBoost
append results

7 feature(s)
Index(['SingalArea_GyrMag_diff', 'IQR_GyrMag_diff', 'SpectralArea_GyrMag_diff',
       'StdDev_GyrMag_diff', 'ZeroCrossing_AccMag_g_diff',
       'DominantFrequency_AccMag_g_diff', 'HarmonicFreq2_AccMag_g_diff'],
      dtype='object')
RF
append results
SVM
append results
MLP


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


append results
KNC
append results
AdaBoost
append results

8 feature(s)
Index(['SingalArea_GyrMag_diff', 'IQR_GyrMag_diff', 'SpectralArea_GyrMag_diff',
       'StdDev_GyrMag_diff', 'ZeroCrossing_AccMag_g_diff',
       'DominantFrequency_AccMag_g_diff', 'HarmonicFreq2_AccMag_g_diff',
       'HarmonicFreq3_AccMag_g_diff'],
      dtype='object')
RF
append results
SVM
append results
MLP


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


append results
KNC
append results
AdaBoost
append results

9 feature(s)
Index(['SingalArea_GyrMag_diff', 'IQR_GyrMag_diff', 'SpectralArea_GyrMag_diff',
       'StdDev_GyrMag_diff', 'ZeroCrossing_AccMag_g_diff',
       'DominantFrequency_AccMag_g_diff', 'HarmonicFreq2_AccMag_g_diff',
       'HarmonicFreq3_AccMag_g_diff', 'SpectralEntropy_AccMag_g_diff'],
      dtype='object')
RF
append results
SVM
append results
MLP


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


append results
KNC
append results
AdaBoost
append results

10 feature(s)
Index(['SingalArea_GyrMag_diff', 'IQR_GyrMag_diff', 'SpectralArea_GyrMag_diff',
       'StdDev_GyrMag_diff', 'ZeroCrossing_AccMag_g_diff',
       'DominantFrequency_AccMag_g_diff', 'HarmonicFreq2_AccMag_g_diff',
       'HarmonicFreq3_AccMag_g_diff', 'SpectralEntropy_AccMag_g_diff',
       'DominantFrequency_GyrMag_diff'],
      dtype='object')
RF
append results
SVM
append results
MLP
